In [2]:
from sfm import SFM
import os, cv2
import numpy as np
import matplotlib.pyplot as plt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [126]:
src = 'datasets/Viking'
detector = cv2.BRISK_create()

In [127]:
files = os.listdir(src)
files.sort()
itoimg = {}
n = 0

for f in files:
    try:
        im = cv2.imread(f"{src}/{f}")
        im.shape

        itoimg[n] = f"{src}/{f}"
        n+=1

    except:
        pass

print(itoimg)

{0: 'datasets/Viking/00.jpg', 1: 'datasets/Viking/01.jpg', 2: 'datasets/Viking/02.jpg', 3: 'datasets/Viking/03.jpg', 4: 'datasets/Viking/04.jpg', 5: 'datasets/Viking/05.jpg', 6: 'datasets/Viking/06.jpg', 7: 'datasets/Viking/07.jpg', 8: 'datasets/Viking/08.jpg', 9: 'datasets/Viking/09.jpg', 10: 'datasets/Viking/10.jpg', 11: 'datasets/Viking/11.jpg', 12: 'datasets/Viking/12.jpg', 13: 'datasets/Viking/13.jpg', 14: 'datasets/Viking/14.jpg', 15: 'datasets/Viking/15.jpg', 16: 'datasets/Viking/16.jpg', 17: 'datasets/Viking/17.jpg', 18: 'datasets/Viking/18.jpg', 19: 'datasets/Viking/19.jpg', 20: 'datasets/Viking/20.jpg', 21: 'datasets/Viking/21.jpg', 22: 'datasets/Viking/22.jpg', 23: 'datasets/Viking/23.jpg', 24: 'datasets/Viking/24.jpg', 25: 'datasets/Viking/25.jpg', 26: 'datasets/Viking/26.jpg', 27: 'datasets/Viking/27.jpg', 28: 'datasets/Viking/28.jpg', 29: 'datasets/Viking/29.jpg', 30: 'datasets/Viking/30.jpg', 31: 'datasets/Viking/31.jpg', 32: 'datasets/Viking/32.jpg', 33: 'datasets/Vikin

In [62]:
def threshold(img, threshold=0):
    img[np.abs(img) > threshold] = 255
    return img

def lowpassFilter(img, kernelSize, sigma=1.0, mu=0.0):
    x, y = np.meshgrid(np.linspace(-1, 1, kernelSize), np.linspace(-1, 1, kernelSize))
    d = np.sqrt(x*x + y*y)
    kernel = np.exp(-((d - mu)**2 / (2.0 * sigma**2)))

    smoothed = cv2.filter2D(img, -1, kernel)
    return smoothed

In [134]:
i = 0

out_dir = 'pres_out'

while True:
    frame = cv2.imread(itoimg[i])
    frame_g = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # g0 = cv2.GaussianBlur(frame_g, (25, 25), 1)
    # g1 = cv2.GaussianBlur(frame_g, (25,25), 0.3)
    # frame_gauss = g1-g0

    frame_gauss = cv2.GaussianBlur(frame_gauss, (7, 7), 0)

    edges = cv2.Canny(frame_gauss, 0, 100)
    # g0 = cv2.GaussianBlur(edges, (25,25), 0)
    # g1 = cv2.GaussianBlur(edges, (25,25), 0)
    # edges = g1-g0
    # edges = cv2.Sobel(frame_g, cv2.CV_64F, dx=1, dy=0, ksize=5)

    _frame = edges
    _frame = lowpassFilter(_frame, 35)
    ret, mask = cv2.threshold(_frame, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    # mask = lowpassFilter(mask, 15)
    mask = cv2.medianBlur(mask, 95)
    mask = cv2.medianBlur(mask, 95)
    out = cv2.bitwise_and(frame, frame, mask=mask)
    kps = detector.detect(frame)
    kps_good = []

    kps_pt = [kp.pt for kp in kps]
    kps_mean_col = [out[np.uint16(kp[1])][np.uint16(kp[0])] for kp in kps_pt]
    removed = 0
    for idx in range(len(kps_mean_col)):
        if not np.any(kps_mean_col[idx]):
            removed+=1
            continue
        kps_good.append(kps[idx])
    out_kps_filt = cv2.drawKeypoints(out, kps_good, None)
    out_kps = cv2.drawKeypoints(frame, kps, None)

    print(np.mean(mask))
    print('removed: ', removed)
    print('total: ', len(kps_good))
    print('prcnt rm: ', removed/len(kps_pt))
    print('score', removed/len(kps_pt) + frame.shape[1]/(10000*np.mean(mask)))

    i+=1

    frame = cv2.resize(frame, (640, 480), interpolation= cv2.INTER_LINEAR)
    out_kps = cv2.resize(out_kps, (640, 480), interpolation= cv2.INTER_LINEAR)
    edges = cv2.resize(edges, (640, 480), interpolation= cv2.INTER_LINEAR)
    mask = cv2.resize(mask, (640, 480), interpolation= cv2.INTER_LINEAR)
    out_kps_filt = cv2.resize(out_kps_filt, (640, 480), interpolation= cv2.INTER_LINEAR)
    frame_gauss = cv2.resize(frame_gauss, (640, 480), interpolation= cv2.INTER_LINEAR)

    cv2.imshow("frame", frame)
    cv2.imshow("detections", out_kps)
    cv2.imshow("Gaussian", frame_gauss)
    cv2.imshow("edges", edges)
    cv2.imshow("mask", mask)
    cv2.imshow("detections filt", out_kps_filt)

    key = cv2.waitKey(0)
    if key == 27:
        cv2.destroyAllWindows()
        break

    if key == ord('s'):
        cv2.imwrite(f'{out_dir}/{i}_source.png', frame)
        cv2.imwrite(f'{out_dir}/{i}_detections.png', out_kps)
        cv2.imwrite(f'{out_dir}/{i}_edges.png', edges)
        cv2.imwrite(f'{out_dir}/{i}_mask.png', mask)
        cv2.imwrite(f'{out_dir}/{i}_detections_filtered.png', out_kps_filt)

    # fix for notebook
    if i == n:
        cv2.destroyAllWindows()
        break


102.99820188492063
removed:  334
total:  1763
prcnt rm:  0.1592751549833095
score 0.15976448391408357
